# LangChain을 사용한 함수 호출 가이드

## LangChain을 함수 호출에 사용하는 이유

LangChain은 함수 호출 기능을 구현하는 데 여러 강력한 이점을 제공한다.

  * **모듈성 (Modularity)**: 도구, 체인, LLM을 마치 레고 블록처럼 쉽게 조합하고 교체할 수 있다.
  * **유연성 (Flexibility)**: 간단한 챗봇부터 복잡한 데이터 분석 에이전트까지 광범위한 사용 사례를 지원한다.
  * **확장성 (Scalability)**: 더 많은 도구와 기능이 추가되어 워크플로우가 복잡해져도 효율적으로 처리하고 관리할 수 있다.
  * **커뮤니티 지원 (Community Support)**: 방대한 문서와 활발한 개발 커뮤니티 덕분에 문제 해결과 기능 확장이 용이하다.

In [13]:
#%pip install --upgrade langchain

In [3]:
import os
from dotenv import load_dotenv  

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

### 핵심 개념

  * **도구 (Tools)**: 에이전트가 호출할 수 있는 함수를 의미한다. 단순히 함수 그 자체가 아니라, LLM이 이해할 수 있도록 **설명이 포함된 함수**의 포장지(wrapper)다.
  * **체인 (Chains)**: 여러 프롬프트나 도구를 순차적으로 결합하여 만드는 워크플로우다. 정해진 순서대로 작업을 처리할 때 유용하다.
  * **에이전트 (Agents)**: LLM을 핵심 두뇌로 사용하여, 사용자 쿼리에 따라 어떤 도구를 사용해야 할지 **스스로 판단하고 행동**하는 주체다.

## 첫 번째 함수 호출 에이전트 만들기

날씨 데이터를 가져오는 간단한 에이전트를 만들고, 점차 기능을 확장해 본다.

### 1단계: 함수 정의하기

날씨 데이터를 가져오는 파이썬 함수를 생성한다. 여기서는 실제 API 호출 대신 간단한 모의(mock) 함수를 사용한다.

In [14]:
import requests

def get_weather(city: str) -> str:
    """주어진 도시의 날씨 정보를 반환한다."""
    # 실제 구현 시에는 이 부분에 날씨 API 호출 코드를 작성한다.
    return f"{city}의 날씨는 맑고 최고 기온은 25°C이다."

### 2단계: 함수를 도구(Tool)로 감싸기

LangChain이 함수를 인식하고 사용할 수 있도록 `Tool` 클래스로 감싸준다. 이 과정에서 `description` 필드는 LLM이 이 도구의 용도를 파악하는 데 결정적인 역할을 한다.

In [15]:
from langchain.tools import Tool

weather_tool = Tool(
    name="get_weather",
    func=get_weather,
    description="특정 도시의 날씨 정보를 가져올 때 사용된다."
)

### 3단계: 에이전트 초기화하기

필요한 도구와 LLM을 지정하여 에이전트를 설정한다.

In [16]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType

# 언어 모델을 초기화한다.
llm = ChatOpenAI(temperature=0)

# 에이전트가 사용할 도구 목록을 정의한다.
tools = [weather_tool]

# 에이전트를 초기화한다.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

### 4단계: 에이전트 테스트하기

에이전트와 상호작용하여 의도대로 작동하는지 확인한다.

In [17]:
response = agent.run("뉴욕의 날씨는 어때?")
print(response)



> Entering new AgentExecutor chain...
I should use the get_weather function to retrieve the weather information for New York.
Action: get_weather
Action Input: "New York"
Observation: New York의 날씨는 맑고 최고 기온은 25°C이다.
Thought:I now have the weather information for New York.
Final Answer: New York의 날씨는 맑고 최고 기온은 25°C이다.

> Finished chain.
New York의 날씨는 맑고 최고 기온은 25°C이다.


### 5단계: 도구 추가하기

사용자 위치를 파악하는 함수와 도구를 추가로 정의한다.

In [18]:
def get_location_from_ip(ip: str) -> str:
    """IP 주소를 기반으로 위치(도시 이름)를 반환한다."""
    # 실제로는 IP 조회 API를 사용한다.
    return "New York"

location_tool = Tool(
    name="get_location_from_ip",
    func=get_location_from_ip,
    description="사용자의 IP 주소를 사용하여 현재 위치를 파악할 때 사용한다."
)

### 6단계: 에이전트 업데이트하기

새로운 도구를 포함하여 에이전트를 다시 설정한다. 이제 에이전트는 두 가지 도구 중 상황에 맞는 것을 선택하거나, 두 도구를 순차적으로 사용할 수 있다.

In [19]:
tools = [weather_tool, location_tool]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

response = agent.run("내 IP 주소 123.45.67.89가 있는 곳의 날씨는 어때?")
print(response)



> Entering new AgentExecutor chain...
우선 사용자의 현재 위치를 파악해야 한다.
Action: get_location_from_ip
Action Input: 123.45.67.89
Observation: New York
Thought:이제 뉴욕의 날씨 정보를 가져와야 한다.
Action: get_weather
Action Input: New York
Observation: New York의 날씨는 맑고 최고 기온은 25°C이다.
Thought:날씨 정보를 알았으므로 최종 답변을 할 수 있다.
Final Answer: 뉴욕의 날씨는 맑고 최고 기온은 25°C이다.

> Finished chain.
뉴욕의 날씨는 맑고 최고 기온은 25°C이다.


이 경우, 에이전트는 먼저 `get_location_from_ip`를 호출하여 'New York'을 얻고, 그 결과를 다시 `get_weather` 도구에 입력하여 최종 날씨 정보를 반환한다.

## 에이전트에 대화 기록(Memory) 추가하기

더 자연스러운 대화형 에이전트를 만들기 위해, 이전 대화 내용을 기억하는 메모리 기능을 통합한다.

### 7단계: 대화 메모리 추가하기

LangChain은 `ConversationBufferMemory`를 통해 단기 기억을 쉽게 구현할 수 있도록 지원한다.

In [21]:
from langchain.memory import ConversationBufferMemory

# 메모리 객체를 초기화한다.
memory = ConversationBufferMemory(memory_key="chat_history")

# 메모리를 포함하여 에이전트를 업데이트한다.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, # 메모리 사용 시에는 CONVERSATIONAL 타입이 적합하다.
    memory=memory,
    verbose=True
)

# 메모리 기능 테스트
response = agent.run("파리의 날씨는 어때?")
print(response)

response = agent.run("내가 방금 무엇을 물어봤는지 다시 알려줄래?")
print(response)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_weather
Action Input: Paris
Observation: Paris의 날씨는 맑고 최고 기온은 25°C이다.
Thought:Do I need to use a tool? No
AI: Paris의 날씨는 맑고 최고 기온은 25°C이에요.

> Finished chain.
Paris의 날씨는 맑고 최고 기온은 25°C이에요.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_weather
Action Input: Paris
Observation: Paris의 날씨는 맑고 최고 기온은 25°C이다.
Thought:Do I need to use a tool? No
AI: You asked about the weather in Paris. It is currently clear with a high of 25°C.

> Finished chain.
You asked about the weather in Paris. It is currently clear with a high of 25°C.


## API 호출 및 오류 처리

실제 애플리케이션에서는 외부 API와 통신하게 된다. `get_weather` 함수를 실제 API를 사용하고 오류를 처리하도록 수정해 본다.

In [22]:
def get_weather(city: str) -> str:
    """실제 날씨 API를 호출하여 날씨 정보를 가져온다."""
    try:
        # 실제 API 엔드포인트와 키를 사용해야 한다.
        response = requests.get(f"https://api.weatherapi.com/v1/current.json?key=YOUR_API_KEY&q={city}")
        response.raise_for_status() # 오류가 발생하면 예외를 일으킨다.
        data = response.json()
        return f"{city}의 날씨는 {data['current']['condition']['text']}이며, 기온은 {data['current']['temp_c']}°C이다."
    except Exception as e:
        return f"오류가 발생했습니다: {str(e)}"